In [ ]:
pip install findspark

In [ ]:
pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
#SparkSession.builder
 #    .master("local")
  #   .appName("Word Count")
   #  .config("spark.some.config.option", "some-value")
   #  .getOrCreate()

spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello ")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [3]:
spark

In [35]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

sql_sc = SQLContext(spark)
repository='C:/Users/ahajjej/Desktop/Adam_Work_Spark/trips_spark/Shared_Micromobility_Vehicle_Trips.csv'
Austin_Trips_Df = spark.read.csv(repository,header='True',inferSchema='true') 


In [42]:
Austin_Trips_Df.count()

2115445

In [36]:
Austin_Trips_Df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Device ID: string (nullable = true)
 |-- Vehicle Type: string (nullable = true)
 |-- Trip Duration: integer (nullable = true)
 |-- Trip Distance: integer (nullable = true)
 |-- Start Time: string (nullable = true)
 |-- End Time: string (nullable = true)
 |-- Modified Date: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Day of Week: integer (nullable = true)
 |-- Council District (Start): integer (nullable = true)
 |-- Council District (End): integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Census Tract Start: string (nullable = true)
 |-- Census Tract End: string (nullable = true)



In [38]:
Austin_Trips_Df.head(1)

[Row(ID='900f1254-8a1e-495e-a55f-1153ebdb3e0e', Device ID='4f7f79d1-88d8-4829-95be-5a548134425b', Vehicle Type='bicycle', Trip Duration=188, Trip Distance=836, Start Time=datetime.datetime(2019, 7, 1, 0, 0), End Time=datetime.datetime(2019, 7, 1, 0, 0), Modified Date='07/03/2019 05:02:16 AM', Month=7, Hour=0, Day of Week=1, Council District (Start)=3, Council District (End)=3, Year=2019, Census Tract Start='48453002111', Census Tract End='48453002111')]

In [51]:
 Austin_Trips_Df = Austin_Trips_Df.dropDuplicates()

In [65]:
Austin_Trips_Df.dropna().count()

2115398

In [37]:
#from pyspark.sql.functions import to_timestamp
#Austin_Trips_Df = Austin_Trips_Df.withColumn('Start_Time',to_timestamp(Austin_Trips_Df['Start Time'], "yyyy_MM_dd HH_mm_ss"))
from pyspark.sql.functions import unix_timestamp
pattern ='MM/dd/yyyy hh:mm:ss aa'
Austin_Trips_Df=Austin_Trips_Df.withColumn('Start Time',unix_timestamp(Austin_Trips_Df['Start Time'],pattern).cast("timestamp"))\
.withColumn('End Time',unix_timestamp(Austin_Trips_Df['End Time'],pattern).cast("timestamp"))

In [39]:
Austin_Trips_Df=Austin_Trips_Df.withColumn('Trip Distance',(Austin_Trips_Df['Trip Distance'])/1000).withColumnRenamed('Trip Distance', 'Trip_Distance(km)').withColumn('Trip Duration',(Austin_Trips_Df['Trip Duration'])/3600).withColumnRenamed('Trip Duration', 'Trip_Duration(H)')

In [40]:
Austin_Trips_Df=Austin_Trips_Df.drop(Austin_Trips_Df['Modified Date']).drop(Austin_Trips_Df['Year'])

In [41]:
Austin_Trips_Df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Device ID: string (nullable = true)
 |-- Vehicle Type: string (nullable = true)
 |-- Trip_Duration(H): double (nullable = true)
 |-- Trip_Distance(km): double (nullable = true)
 |-- Start Time: timestamp (nullable = true)
 |-- End Time: timestamp (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Day of Week: integer (nullable = true)
 |-- Council District (Start): integer (nullable = true)
 |-- Council District (End): integer (nullable = true)
 |-- Census Tract Start: string (nullable = true)
 |-- Census Tract End: string (nullable = true)



In [47]:
Austin_Trips_Df.select("Trip_Duration(H)","Trip_Distance(km)").describe().show(truncate= True)

+-------+-------------------+-----------------+
|summary|   Trip_Duration(H)|Trip_Distance(km)|
+-------+-------------------+-----------------+
|  count|            2115445|          2115445|
|   mean|0.16955681625694233|45.60924962360066|
| stddev| 0.2429257070840963| 1936.53659027175|
|    min|             -0.315|              0.0|
|    max|            23.5475|      1113117.831|
+-------+-------------------+-----------------+



In [49]:
Austin_Trips_Df=Austin_Trips_Df.filter(Austin_Trips_Df['Trip_Duration(H)']>0)

In [50]:
Austin_Trips_Df.count()

2115406

In [62]:
Austin_Trips_Df.crosstab('Vehicle Type', 'Month').show()

+------------------+------+---+------+------+------+
|Vehicle Type_Month|    10| 11|     7|     8|     9|
+------------------+------+---+------+------+------+
|           scooter|503906| 88|471763|471527|557695|
|           bicycle| 26521|  0| 31633| 25036| 27237|
+------------------+------+---+------+------+------+



In [63]:
Austin_Trips_Df.crosstab('Day of Week', 'Month').show()

+-----------------+------+---+-----+------+------+
|Day of Week_Month|    10| 11|    7|     8|     9|
+-----------------+------+---+-----+------+------+
|                0| 78276|  0|67526| 60053| 92386|
|                5| 82200| 88|83536| 94825| 91179|
|                1| 58880|  0|57008| 45884| 76615|
|                6|101599|  0|96908|118696|126083|
|                2| 67055|  0|62289| 49164| 61974|
|                3| 67917|  0|65831| 55822| 65591|
|                4| 74500|  0|70298| 72119| 71104|
+-----------------+------+---+-----+------+------+



In [70]:
Austin_Trips_Df=Austin_Trips_Df.withColumn('Speed',Austin_Trips_Df['Trip_Distance(km)']/Austin_Trips_Df['Trip_Duration(H)'])

In [71]:
Austin_Trips_Df.describe(['Speed']).show()

+-------+-----------------+
|summary|            Speed|
+-------+-----------------+
|  count|          2115406|
|   mean|431.7605969129722|
| stddev|17208.99638077851|
|    min|              0.0|
|    max|5242221.507692307|
+-------+-----------------+



In [110]:
SpeedDf=Austin_Trips_Df.select("Trip_Distance(km)","Trip_Duration(H)","Speed")
#.groupBy("Speed").count().orderBy("Speed")
#SpeedDf.describe().show()
#ScatterChart(plot,fields=Some(("Speed","Count")))
#import matplotlib.pyplot as plt
#plot1=SpeedDf.toPandas()


def calculate_bounds(SpeedDf):
                bounds = {
                        c: dict(
                            zip(["q1", "q3"], SpeedDf.approxQuantile(c, [0.25, 0.75], 0))
                        )
                        for c,d in zip(SpeedDf.columns, SpeedDf.dtypes) if d[1] == "int"
                    }
        
            for c in bounds:
                            iqr = bounds[c]['q3'] - bounds[c]['q1']
                            bounds[c]['min'] = bounds[c]['q1'] - (iqr * 1.5)
                            bounds[c]['max'] = bounds[c]['q3'] + (iqr * 1.5)

                 return bounds

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 17)

In [112]:
SpeedDf=Austin_Trips_Df.select("Trip_Distance(km)","Trip_Duration(H)","Speed")
bounds = {
    c: dict(
        zip(["q1", "q3"], SpeedDf.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c in SpeedDf.columns
}

In [113]:
for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
    bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)
print(bounds)

{'Trip_Distance(km)': {'q1': 0.598, 'q3': 2.037, 'lower': -1.5605000000000002, 'upper': 4.1955}, 'Trip_Duration(H)': {'q1': 0.06555555555555556, 'q3': 0.1922222222222222, 'lower': -0.12444444444444441, 'upper': 0.3822222222222222}, 'Speed': {'q1': 6.252631578947369, 'q3': 14.195876288659795, 'lower': -5.66223548562127, 'upper': 26.110743353228436}}


In [116]:
import pyspark.sql.functions as f
SpeedDf.select(
    "*",
    *[
        f.when(
            f.col(c).between(bounds[c]['lower'], bounds[c]['upper']),
            0
        ).otherwise(1).alias(c+"_out") 
        for c in SpeedDf.columns
    ]
).show(20)

+-----------------+--------------------+------------------+---------------------+--------------------+---------+
|Trip_Distance(km)|    Trip_Duration(H)|             Speed|Trip_Distance(km)_out|Trip_Duration(H)_out|Speed_out|
+-----------------+--------------------+------------------+---------------------+--------------------+---------+
|            0.768| 0.14083333333333334|5.4532544378698224|                    0|                   0|        0|
|            5.536|  0.3638888888888889|15.213435114503817|                    1|                   0|        0|
|              1.3| 0.12472222222222222|10.423162583518932|                    0|                   0|        0|
|            3.619| 0.22583333333333333|16.025092250922512|                    0|                   0|        0|
|            1.335| 0.15777777777777777| 8.461267605633804|                    0|                   0|        0|
|            1.154| 0.14944444444444444| 7.721933085501859|                    0|               